Inputs and definitions

In [122]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

price_chaos_per_ex = 113.0

def strip_digits(input_string):
    return ''.join([i for i in input_string if i.isalpha() or i==' '])

def strip_alpha(input_string):
    return ''.join([i for i in input_string if i.isnumeric() or i==' ' or i=='.'])

def convert_rolls(input_string):
    return [np.mean([float(roll) for roll in str.split(input_string)])]

Read database

In [123]:
with open('.env-postgres') as f:
    lines = f.readlines()
engine = create_engine('postgresql://' + lines[0] + '@localhost:5432/poeitems')
df=pd.read_sql_query('select * from items',con=engine)
df.dropna(subset='explicit', axis='rows', inplace=True)

subset_ex = df['price'].str.contains('exalted')
subset_c = df['price'].str.contains('chaos')
df = df[subset_c|subset_ex]

subset_ex = df['price'].str.contains('exalted')
df['price'] = df['price'].str.replace('~price ','')
df['price'] = df['price'].str.replace(' exalted','')
df['price'] = df['price'].str.replace(' chaos','')
df['price'] = df['price'].str.replace('~b/o ','')
df['price'] = df['price'].replace('', np.nan)
df['price'] = df['price'].dropna()
df['price'] = df['price'].astype(float)
df.loc[subset_ex, 'price'] = df.loc[subset_ex, 'price']*price_chaos_per_ex

Create explicit lexicon

In [124]:
explicit_mod_dict = {'mod':['']}
explicit_mods = pd.DataFrame.from_dict(explicit_mod_dict)
for _, item in df.iterrows():
    for mod in item.explicit:
        mod_no_digits = strip_digits(mod)
        if mod_no_digits not in explicit_mods.values:
            explicit_mods.loc[len(explicit_mods.index)] = mod_no_digits

explicit_mods.sort_values(by='mod', inplace=True)
explicit_mods['mod'].replace('',np.nan, inplace=True)
explicit_mods.dropna(inplace=True)
print(explicit_mods['mod'].values)

[' Life gained for each Enemy hit by your Attacks'
 ' Life gained for each Enemy hit by your Spells' ' Life gained on Kill'
 ' Mana gained for each Enemy Hit by your Spells'
 ' Mana gained for each Enemy hit by your Attacks' ' Mana gained on Kill'
 ' chance to gain a Frenzy Charge on Kill'
 ' chance to gain a Power Charge on Kill'
 ' chance to gain an Endurance Charge on Kill' ' increased Area of Effect'
 ' increased Area of Effect of Hex Skills' ' increased Armour'
 ' increased Attack Speed'
 ' increased Attack Speed if youve been Hit Recently'
 ' increased Bleeding Duration' ' increased Cast Speed'
 ' increased Chaos Damage' ' increased Cold Damage' ' increased Damage'
 ' increased Damage with Bleeding' ' increased Damage with Poison'
 ' increased Effect of NonDamaging Ailments'
 ' increased Elemental Damage with Attack Skills'
 ' increased Evasion Rating' ' increased Experience gain'
 ' increased Fire Damage' ' increased Global Accuracy Rating'
 ' increased Global Critical Strike Ch

Create implicit lexicon

In [125]:
implicit_mod_dict = {'mod':['']}
implicit_mods = pd.DataFrame.from_dict(implicit_mod_dict)
for _, item in df.iterrows():
    for mod in item.implicit:
        mod_no_digits = strip_digits(mod)
        if mod_no_digits not in implicit_mods.values:
            implicit_mods.loc[len(implicit_mods.index)] = mod_no_digits

implicit_mods.sort_values(by='mod', inplace=True)
implicit_mods['mod'].replace('',np.nan, inplace=True)
implicit_mods.dropna(inplace=True)
print(implicit_mods)

                                                   mod
48                         Fire Damage taken from Hits
32      Life gained for each Enemy hit by your Attacks
53                                 Life gained on Kill
61                    Lightning Damage taken from Hits
62      Mana gained for each Enemy hit by your Attacks
..                                                 ...
103                           Recover  of Life on Kill
106                        Regenerate  Mana per second
37   Regenerate  of Life per second per Endurance C...
17                    Wrath has  increased Aura Effect
79                 Zealotry has  increased Aura Effect

[107 rows x 1 columns]


Merge explicit/implicit lexica into affix lexica

In [126]:
info_list = ['itemid', 'price', 'basetype', 'ilvl']
explicit_list = explicit_mods['mod'].tolist()
explicit_list = [s + ' (explicit)' for s in explicit_list]
implicit_list = implicit_mods['mod'].tolist()
implicit_list = [s + ' (implicit)' for s in implicit_list]
df_affix = pd.DataFrame(columns=info_list + explicit_list + implicit_list)
print(df_affix.head())

Empty DataFrame
Columns: [itemid, price, basetype, ilvl,  Life gained for each Enemy hit by your Attacks (explicit),  Life gained for each Enemy hit by your Spells (explicit),  Life gained on Kill (explicit),  Mana gained for each Enemy Hit by your Spells (explicit),  Mana gained for each Enemy hit by your Attacks (explicit),  Mana gained on Kill (explicit),  chance to gain a Frenzy Charge on Kill (explicit),  chance to gain a Power Charge on Kill (explicit),  chance to gain an Endurance Charge on Kill (explicit),  increased Area of Effect (explicit),  increased Area of Effect of Hex Skills (explicit),  increased Armour (explicit),  increased Attack Speed (explicit),  increased Attack Speed if youve been Hit Recently (explicit),  increased Bleeding Duration (explicit),  increased Cast Speed (explicit),  increased Chaos Damage (explicit),  increased Cold Damage (explicit),  increased Damage (explicit),  increased Damage with Bleeding (explicit),  increased Damage with Poison (explicit),

Parse DB into feature matrix

In [132]:
df_temp3 = df_affix

def item_parser(input_item):
    global df_temp3
    if input_item['itemid'] in df_temp3['itemid']:
        pass
    item_dict = {}
    item_dict['itemid'] = input_item.loc['itemid']
    item_dict['price'] = input_item.loc['price']
    item_dict['basetype'] = input_item.loc['basetype']
    item_dict['ilvl'] = input_item.loc['ilvl']
    for mod in input_item.loc['implicit']:
        affix = strip_digits(mod) + ' (implicit)'
        value = strip_alpha(mod)
        item_dict[affix] = convert_rolls(value)
    for mod in input_item.loc['explicit']:
        affix = strip_digits(mod) + ' (explicit)'
        value = strip_alpha(mod)
        item_dict[affix] = convert_rolls(value)
    df_item = pd.DataFrame.from_dict(item_dict)
    df_temp3 = df_temp3.merge(df_item, how='outer').fillna(0)
    return True

df_temp4 = df
print(df_temp4.apply(func=item_parser, axis=1))

c:\Users\hauke\PycharmProjects\poescraper\.venv\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\hauke\PycharmProjects\poescraper\.venv\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\hauke\PycharmProjects\poescraper\.venv\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\hauke\PycharmProjects\poescraper\.venv\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\hauke\PycharmProjects\poescraper\.venv\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\hauke\PycharmProjects\poescraper\.venv\lib\site-pac

0         True
1         True
2         True
3         True
4         True
          ... 
162096    True
162125    True
162134    True
162135    True
162136    True
Length: 60236, dtype: bool
